# numba

numba es un compilador de Python a LLVM, que funciona para un subconjunto de la funcionalidad del lenguaje orientado a código científico en el que predomina la manipulación de arrays. Este notebook está basado en nuestro artículo [cómo acelerar código Python con numba](http://pybonacci.org/2015/03/13/como-acelerar-tu-codigo-python-con-numba/)

En la web puedes ver la [evolución del rendimiento](http://numba.pydata.org/numba-benchmark/) de algunos benchmarks gracias a [asv](http://github.com/spacetelescope/asv/).

## ¿Cómo funciona?

http://numba.pydata.org/numba-doc/0.18.2/developer/architecture.html

## Guion

1. Ejemplo trivial: numba.jit
2. Especificando tipos
3. Código que no se acelera: numba.njit
4. Placa plana

## Ejemplo trivial

In [1]:
import numpy as np
import numba

In [2]:
def sum2d(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

a = np.arange(9).reshape(3,3)
print(sum2d(a))

36.0


In [3]:
big_arr = np.random.randn(1000, 1000)

In [4]:
%timeit sum2d(big_arr)

1 loops, best of 3: 232 ms per loop


In [5]:
sum2d_jit = numba.jit(sum2d)

In [6]:
sum2d_jit(a)

36.0

In [7]:
%timeit sum2d_jit(big_arr)

The slowest run took 69.00 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 1.15 ms per loop


## Especificando tipos

Al utilizar `numba.jit` estamos pidiendo a numba que *infiera* los tipos de las variables en el momento de la primera llamada.

http://numba.pydata.org/numba-doc/0.18.2/reference/compilation.html#numba.jit

Podemos ver esto utilizando el método `inspect_types` de la función resultante:

In [8]:
sum2d_jit.inspect_types()

sum2d (array(float64, 2d, C, nonconst),)
--------------------------------------------------------------------------------
# File: <ipython-input-2-884e457c72df>
# --- LINE 1 --- 

def sum2d(arr):

    # --- LINE 2 --- 
    # label 0
    #   arr.1 = arr  :: array(float64, 2d, C, nonconst)
    #   del arr
    #   $0.2 = getattr(attr=shape, value=arr.1)  :: (int64 x 2)
    #   $0.5 = exhaust_iter(count=2, value=$0.2)  :: (int64 x 2)
    #   del $0.2
    #   $0.3 = static_getitem(value=$0.5, index=0)  :: int64
    #   $0.4 = static_getitem(value=$0.5, index=1)  :: int64
    #   del $0.5
    #   M = $0.3  :: int64
    #   del $0.3
    #   N = $0.4  :: int64
    #   del $0.4

    M, N = arr.shape

    # --- LINE 3 --- 
    #   $const0.6 = const(float, 0.0)  :: float64
    #   result = $const0.6  :: float64
    #   del $const0.6

    result = 0.0

    # --- LINE 4 --- 
    #   jump 21
    # label 34
    #   $34.2 = iternext(value=$phi34.1)  :: pair<int64, bool>
    #   $34.3 = pair_first(valu

Como hemos llamado a la función con arrays de enteros y de flotantes, vemos que se han generado las dos especificaciones. No obstante, nosotros podemos forzar los tipos de entrada:

In [9]:
@numba.jit('float64(float64[:, :])')
def sum2d_jit_float(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [10]:
sum2d_jit_float(a.astype(float))

36.0

## Algunas limitaciones

Hay muchas partes de Python que no están implementadas en las versiones actuales de numba: en algunas de ellas se está trabajando activamente, mientras que otras es complicado que vean la luz en un futuro cercano.

Una de estas limitaciones es que no podemos crear arrays nuevos dentro de las funciones. Si lo hacemos, numba no será capaz de optimizar todo el código e irá mucho más lento de lo que esperamos.

In [11]:
@numba.jit
def sum2d_jit_big():
    arr = np.random.randn(1000, 1000)
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [12]:
%timeit sum2d_jit_big()

The slowest run took 4.81 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 56 ms per loop


In [13]:
%timeit sum2d_jit_big.py_func()

1 loops, best of 3: 290 ms per loop


In [14]:
sum2d_jit_big.inspect_types()

sum2d_jit_big ()
--------------------------------------------------------------------------------
# File: <ipython-input-11-ca6a1125999c>
# --- LINE 1 --- 

@numba.jit

# --- LINE 2 --- 

def sum2d_jit_big():

    # --- LINE 3 --- 
    # label 0
    #   $0.1 = global(np: <module 'numpy' from '/home/juanlu/.miniconda3/envs/py34/lib/python3.4/site-packages/numpy/__init__.py'>)  :: pyobject
    #   $0.2 = getattr(attr=random, value=$0.1)  :: pyobject
    #   del $0.1
    #   $0.3 = getattr(attr=randn, value=$0.2)  :: pyobject
    #   del $0.2
    #   $const0.4 = const(int, 1000)  :: pyobject
    #   $const0.5 = const(int, 1000)  :: pyobject
    #   $0.6 = call $0.3($const0.4, $const0.5, )  :: pyobject
    #   del $const0.5
    #   del $const0.4
    #   del $0.3
    #   arr = $0.6  :: pyobject
    #   del $0.6

    arr = np.random.randn(1000, 1000)

    # --- LINE 4 --- 
    #   $0.8 = getattr(attr=shape, value=arr)  :: pyobject
    #   $0.11 = exhaust_iter(count=2, value=$0.8)  :: pyobjec

Al contrario de lo que teníamos antes, se han creado numerosas variables de tipo `pyobject`. Esto sucede en varios casos:

* El algoritmo de inferencia de tipos falla
* Utilizamos características del lenguaje no soportadas por numba
* Utilizamos objetos de Python complejos

## Entendiendo numba: el modo *nopython*

Como podemos leer en la documentación, [numba tiene dos modos de funcionamiento básicos](http://numba.pydata.org/numba-doc/0.17.0/user/jit.html#nopython): el modo *object* y el modo *nopython*.

* El modo *object* genera código que gestiona todas las variables como objetos de Python y utiliza la API C de Python para operar con ellas. En general en este modo **no habrá ganancias de rendimiento** (e incluso puede ir más lento), con lo cual mi recomendación personal es directamente no utilizarlo. Hay casos en los que numba puede detectar los bucles y optimizarlos individualmente (*loop-jitting*), pero no le voy a prestar mucha atención a esto.
* El modo *nopython* **genera código independiente de la API C de Python**. Esto tiene la desventaja de que no podemos usar todas las características del lenguaje, **pero tiene un efecto significativo en el rendimiento**. Otra de las restricciones es que **no se puede reservar memoria para objetos nuevos**.

Por defecto numba usa el modo *nopython* siempre que puede, y si no pasa a modo *object*. Nosotros vamos a **forzar el modo nopython** (o «modo estricto» como me gusta llamarlo) porque es la única forma de aprovechar el potencial de numba.

## Ámbito de aplicación

El problema del modo *nopython* es que los mensajes de error son totalmente inservibles en la mayoría de los casos, así que antes de lanzarnos a compilar funciones con numba conviene hacer un repaso de qué no podemos hacer para anticipar la mejor forma de programar nuestro código. Podéis consultar en la documentación [el subconjunto de Python soportado por numba](http://numba.pydata.org/numba-doc/0.17.0/reference/pysupported.html) en modo *nopython*, y ya os aviso que, al menos de momento, no tenemos [*list comprehensions*](https://github.com/numba/numba/issues/504), [generadores](https://github.com/numba/numba/issues/984) ni algunas cosas más. Permitidme que resalte una frase sacada de la página principal de numba:

> "*With a few annotations, **array-oriented and math-heavy Python code** can be just-in-time compiled to native machine instructions, similar in performance to C, C++ and Fortran*". [Énfasis mío]

Siento decepcionar a la audiencia pero *numba no acelerará todo el código Python* que le echemos: está enfocado a operaciones matemáticas con arrays. Aclarado este punto, vamos a ponernos manos a la obra con un ejemplo aplicado :)

### Solución de Navier de una placa plana

In [33]:
from numpy import sin, pi

def plate_displacement_py(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D))

In [29]:
plate_displacement_nb = numba.jit(plate_displacement_py, nopython=True)

In [30]:
# Plate geometry
a = 1.0  # m
b = 1.0  # m
h = 50e-3  # m

# Material properties
E = 69e9  # Pa
nu = 0.35

# Series terms
max_m = 16
max_n = 16

# Computation points
# NOTE: With an odd number of points the center of the place is included in
# the grid
NUM_POINTS = 101
max_i = NUM_POINTS
max_j = NUM_POINTS

# Load
P = 10e3  # N
xi = a / 2
eta = a / 2

# Flexural rigidity
D = h**3 * E / (12 * (1 - nu**2))

# ---

# Set up domain
x = np.linspace(0, a, num=NUM_POINTS)
y = np.linspace(0, b, num=NUM_POINTS)
xx, yy = np.meshgrid(x, y)


In [31]:
# Compute displacement field
ww = np.zeros_like(xx)
plate_displacement_nb(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

# Print maximum displacement
w_max = np.abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


In [32]:
%timeit plate_displacement_nb(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

1 loops, best of 3: 1.13 s per loop


**Conseguimos multiplicar por 100 el rendimiento**, aunque como ya hemos visto este no es nuestro mejor resultado.

Si probamos la versión de NumPy las cosas no mejoran respecto al original, porque se lanza el modo `object`:

In [34]:
def plate_displacement_np(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
            ww += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                   * sin(mm * pi * xx / a)
                   * sin(nn * pi * yy / b)
                   / (pi**4 * D))

plate_displacement_nb2 = numba.jit(plate_displacement_np)

In [35]:
# Compute displacement field
ww = np.zeros_like(xx)
plate_displacement_nb2(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

# Print maximum displacement
w_max = np.abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


In [36]:
%timeit plate_displacement_nb2(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

1 loops, best of 3: 205 ms per loop


**El rendimiento es igual al de NumPy, sin más**.

## Conclusiones

numba es una herramienta muy poderosa con la que podemos exprimir el rendimiento del código haciendo muy pocos cambios y todavía escribiendo en un único lenguaje. Hay otras características poderosas, como liberación del GIL o aceleración en GPUs, que no vamos a ver aquí. No obstante, aunque la API se ha estabilizado bastante en las últimas versiones aún quedan algunas regresiones de rendimiento por encontrar y características del lenguaje por implementar. El desarrollo es muy rápido así que seguiremos viendo novedades en este frente durante los próximos meses.